In [1]:
# imports
import os
import sys

# code for enabling this notebook to work within cursor
coralme_dir = '../'#/home/chris/zuniga/coralme/'
sys.path.insert(0, coralme_dir)

import cobra
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
from coralme.util.helpers import get_nlp, optimize
import pickle
from copy import deepcopy
from concurrent.futures import ProcessPoolExecutor, as_completed

def run_KOd_gene(zerod_rxns, nlp, rxn_index_dct, met_index_dct, basis, save_path):
    # set the reacitons to zero, save old values
    for rxn in zerod_rxns:
        nlp.xl[rxn_index_dct[rxn]] = lambda x : 0
        nlp.xu[rxn_index_dct[rxn]] = lambda x : 0

    # run the model
    sol, _ = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.5, min_mu = 0.01, maxIter = 20,
		tolerance = 1e-4, precision = 'quad', verbose = True, basis = basis)

    # save the results
    pickle_out = open(save_path, 'wb')
    pickle.dump(sol, pickle_out)
    pickle_out.close()

In [9]:
# loop through every organism
base_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files')
for org in os.listdir(os.path.join(base_dir, 'individual_species')):
    if 'Reference' in org: continue
    org_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files', 'individual_species', org)
    print('running on '+org+' : ', end = '')
    
    # check if run
    if os.path.exists(os.path.join(org_dir, 'outputs', 'all_genes_KOd.pkl')):
        print('already run!')
        continue
    print('loading...', end = '')
    # load model
    path = os.path.join(org_dir, 'outputs', 'pathway_annotated_model.pkl')
    if not os.path.exists(path):
        print('no genome annotated model exists, skipping')
        continue
    ME_model = coralme.io.pickle.load_pickle_me_model(path)

    # create directories
    if not os.path.exists(os.path.join(org_dir, 'outputs', 'gene_KO_solutions')):
        os.mkdir(os.path.join(org_dir, 'outputs', 'gene_KO_solutions'))
    
    # transforn into nlp
    nlp = get_nlp(ME_model)
    rxn_index_dct = {r.id : ME_model.reactions.index(r) for r in ME_model.reactions}
    met_index_dct = {m.id : ME_model.metabolites.index(m) for m in ME_model.metabolites}
    
    # generate basis for later runs
    print('generating basis...', end = '')
    sol, basis = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.5, min_mu = 0.01, maxIter = 20,
    		tolerance = 1e-4, precision = 'quad', verbose = True, basis = None)
    
    if not sol:
        # the basis didn't succeed so removing genes won't help, let's move on
        pickle_out = open(os.path.join(org_dir, 'outputs', 'all_genes_KOd.pkl'), 'wb')
        pickle.dump(False, pickle_out)
        pickle_out.close()
        print(' no basis, so no point in KOing, moving on')
        continue
    else:
        pickle_out = open(os.path.join(org_dir, 'outputs', 'all_genes_KOd.pkl'), 'wb')
    
    # collect all genes
    print('generating inputs...', end = '')
    gene_to_rxns = {}
    for rxn in ME_model.reactions:
        if 'genes' not in rxn.annotation:
            continue
        for gene in rxn.annotation['genes']:
            if gene not in gene_to_rxns:
                gene_to_rxns.update({gene : set()})
            gene_to_rxns[gene].add(rxn.id)
    
    # generate inputs for multithreading
    inputs = []
    KOd_genes = []
    for gene, rxns in gene_to_rxns.items():
        # inputs
        temp_nlp = deepcopy(nlp)
        save_path = os.path.join(org_dir, 'outputs', 'gene_KO_solutions', gene+'.pkl')
        if os.path.exists(save_path):
            continue
    
        inputs.append((rxns, temp_nlp, rxn_index_dct, met_index_dct, basis, save_path))
        KOd_genes.append(gene)
    
    # run
    print('multiprocessing...', end = '')
    with ProcessPoolExecutor() as executor:
        future_to_KOd = {executor.submit(run_KOd_gene, *input_vals) : KOd_gene for input_vals, KOd_gene in zip(inputs, KOd_genes)}
        
        for future in as_completed(future_to_KOd):
            change = future_to_KOd[future]
            try:
                result = future.result()
                print(f"Completed optimization for gene KO {change}")
            except Exception as e:
                print(f"Failed optimization for gene KO {change}: {str(e)}")


    # save a check that all are run
    pickle_out = open(os.path.join(org_dir, 'outputs', 'all_genes_KOd.pkl'), 'wb')
    pickle.dump(True, pickle_out)
    pickle_out.close()
    print(' done!')

running on CP014784 : loading...generating basis...Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.2550000000000000	Not feasible
        2	0.1325000000000000	Not feasible
        3	0.0712500000000000	Not feasible
        4	0.0406250000000000	Not feasible
        5	0.0253125000000000	Not feasible
        6	0.0176562500000000	Not feasible
        7	0.0138281250000000	Not feasible
        8	0.0119140625000000	Not feasible
        9	0.0109570312500000	Not feasible
       10	0.0104785156250000	Not feasible
       11	0.0102392578125000	Not feasible
       12	0.0101196289062500	Not feasible
       13	0.0100598144531250	Not feasible
 no basis, so no point in KOing, moving on
running on CP061848 : loading...generating basis...Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.2550000000000000	Not feasible
        2	0.1325000000000000	Not feasible
        3	0.0712500000000000	Not feasible
        4	0

NameError: name 'fo' is not defined